In [1]:
import mediapipe as mp
import cv2
import os
import numpy as np
import skimage as ski

Extract from video

In [3]:
cam = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print("Ignoring empty camera frame.")
            continue

        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False
        results = hands.process(frame)

        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS) 
                    # ,mp_drawing_styles.get_default_hand_landmarks_style(),
                    # mp_drawing_styles.get_default_hand_connections_style())
        cv2.imshow('MediaPipe Hands', frame)
        if cv2.waitKey(5) & 0xFF == 27: # Press esc to quit
            cv2.destroyAllWindows()
            break
        

               
cam.release()

Extract from picture

In [46]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    original_frame = cv2.imread("raw_data/palms_test.jpg")

    frame = cv2.cvtColor(cv2.flip(original_frame, 1), cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = hands.process(frame)

    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS) 
                # ,mp_drawing_styles.get_default_hand_landmarks_style(),
                # mp_drawing_styles.get_default_hand_connections_style())
                
cv2.imshow('MediaPipe Hands', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
        

               


In [42]:
print(results.multi_hand_landmarks[0].landmark[0], results.multi_hand_landmarks[1].landmark[0])

x: 0.0830296
y: 0.773183227
z: 6.02224134e-007
 x: 0.954928875
y: 0.875744
z: 5.91532171e-007



Verify with sobel, prewitt, canny

In [43]:
gray_frame = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)


# Sobel
sobel = ski.filters.sobel(gray_frame)

# Canny
canny = ski.feature.canny(gray_frame, low_threshold = 100, high_threshold = 200)

# Prewitt
prewitt = ski.filters.prewitt(gray_frame)

cv2.imshow('Sobel', sobel) # Less computational cost
cv2.imshow('Canny', canny) # More precise
cv2.imshow('Prewitt', prewitt)
cv2.moveWindow("Prewitt", 100, 200)

cv2.waitKey(0)
cv2.destroyAllWinºdows()

eliminate noise with mog

In [45]:
def read_video(videopath):
    """
    Reads a video file and returns its frames along with video properties.

    Args:
        videopath (str): The path to the video file.

    Returns:
        tuple: A tuple containing:
            - frames (list): A list of frames read from the video.
            - frame_width (int): The width of the video frames.
            - frame_height (int): The height of the video frames.
            - frame_rate (float): The frame rate of the video.
    """

    #TODO: Complete this line to read the video file
    cap = cv2.VideoCapture(videopath) 
    
    #TODO: Check if the video was successfully opened
    if not cap.isOpened():
        print('Error: Could not open the video file')

    #TODO: Get the szie of frames and the frame rate of the video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Get the width of the video frames
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Get the height of the video frames
    frame_rate = cap.get(cv2.CAP_PROP_FPS) # Get the frame rate of the video
    
    #TODO: Use a loop to read the frames of the video and store them in a list
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frames.append(frame)
        
    cap.release()
    
    return frames, frame_width, frame_height, int(frame_rate)

In [7]:
cam = cv2.VideoCapture(0)

output_folder = "processed_data/mog"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
histories = [200]  # Number of frames to use to build the background model
nmixtures = [7]  # Number of Gaussian mixtures
background_ratios = [0.5]  # Background ratio to consider a pixel as background

for history in histories: 
    for mixture in nmixtures:
        for ratio in background_ratios:
            
            mog = cv2.bgsegm.createBackgroundSubtractorMOG(history, mixture, ratio, 0) # 0 means an automatic noise reduction    

            # Name of the output video file with the parameters (history, varThreshold, detectShadows)
            videoname = f'mog_{history}_{mixture}_{ratio}.mp4' # Name of the output video file with the parameters
            videoname = os.path.join(output_folder, videoname)
            
            # Create a VideoWriter object to save the video
            fourcc = cv2.VideoWriter_fourcc(*"mp4v") # Codec to use
            frame_size = (int(cam.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))) # Size of the frames
            fps = cam.get(cv2.CAP_PROP_FPS) # Frame rate of the video
            out_mog = cv2.VideoWriter(videoname, fourcc, fps, frame_size)
            out_canny = cv2.VideoWriter(output_folder+'/canny.mp4', fourcc, fps, frame_size)
            out_sobel = cv2.VideoWriter(output_folder+'/sobel.mp4', fourcc, fps, frame_size)

            while cam.isOpened():
                ret, frame = cam.read()
                if not ret:
                    break
                    

                # Apply the MOG algorithm to detect the moving objects
                mask = mog.apply(frame)
                # Convert to BGR the mask to store it in the video
                mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

                # Save the mask in a video
                out_mog.write(mask)

                canny_processed = ski.feature.canny(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)) # , mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                canny_processed = cv2.cvtColor((canny_processed*255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
                sobel_processed = ski.filters.sobel(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)) # , mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                sobel_processed = cv2.cvtColor((sobel_processed * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
                out_canny.write(canny_processed)
                out_sobel.write(sobel_processed)

                cv2.imshow('MOG Mask', mask)
                cv2.imshow('Canny', canny_processed)
                cv2.imshow('Sobel', sobel_processed)
                
                if cv2.waitKey(5) & 0xFF == 27:  # Press esc to quit
                    break

            out_mog.release()
            out_canny.release()
            out_sobel.release()
            
            
cam.release()
cv2.destroyAllWindows()